In [40]:
import numpy as np
import random
import pandas as pd

'''==================================================
Initial set up
=================================================='''

#Hyperparameters
SMALL_ENOUGH = 0.005
GAMMA = 0.9         
NOISE = 0.1

#Define all states
all_states=[]
for i in range(4):
    for j in range(6):
            all_states.append((i,j))

#Define rewards for all states
rewards = {}
for i in all_states:
    if i == (1,5):
        rewards[i] = 1
    elif i == (3,4):
        rewards[i] = -1
    elif i == (3,5):
        rewards[i] = -1
    else:
        rewards[i] = 0

#Dictionnary of possible actions. We have three "end" states (1,5 and 3,4 and 3,5)
actions = {
    (0,0):('D', 'R'), 
    (0,1):('D', 'R', 'L'),    
    (0,2):('D', 'L', 'R'),
    (0,3):('D', 'L', 'R'),
    (0,4):('D', 'L', 'R'),
    (0,5):('D', 'L'),
    (1,0):('D', 'U', 'R'),
    (1,1):('D', 'R', 'L', 'U'),
    (1,2):('D', 'R', 'L', 'U'),
    (1,3):('D', 'R', 'L', 'U'),
    (1,4):('D', 'R', 'L', 'U'),
    (2,0):('D', 'U', 'R'),
    (2,1):('D', 'R', 'L', 'U'),
    (2,2):('D', 'R', 'L', 'U'),
    (2,3):('D', 'R', 'L', 'U'),
    (2,4):('D', 'R', 'L', 'U'),
    (2,5):('D', 'L', 'U'),
    (3,0):('U', 'R'),
    (3,1):('R', 'L', 'U'),
    (3,2):('R', 'L', 'U'),
    (3,3):('R', 'L', 'U'),
    } #I'm not sure if this will work but its certainly a better organization of data

#Define an initial policy
policy={}
for s in actions.keys():
    policy[s] = np.random.choice(actions[s])

#Define initial value function 
V={}
for s in all_states:
    if s in actions.keys():
        V[s] = 0
    if s ==(3,4):
        V[s]=-1
    if s == (3,5):
        V[s]=-1
    if s == (1,5):
        V[s]=1
#By the end of this, the grid, values, and rewards have been set up
#An initial policy has been established, which I guess will be a part of the actions taken


In [41]:

def one_trial():
    Terminal = False
    trial_list = []
    start=random.choice([(0,0),(2,1)])
    state = start
    trial_list.append(state)

    while not Terminal:
        #choose an action randomly from the permissible actions
        a = random.choice(actions[state])
        ex = state[0]
        why = state[1]
        if a == 'U':
            nxt = (ex-1, why)
        if a == 'D':
            nxt = (ex+1, why)
        if a == 'L':
            nxt = (ex, why-1)
        if a == 'R':
            nxt = (ex, why+1)
        trial_list.append(nxt)

        #now, a random state has been chosen and the next state established
        value = V[nxt]

        if value == 1:
            Terminal = True
            break
        elif value == -1:
            Terminal = True
            break
        state = nxt
    return trial_list


In [57]:
five_trials = [0]*5
five_trials[0] = one_trial()
five_trials[1]  = one_trial()
five_trials[2]  = one_trial()
five_trials[3]  = one_trial()
five_trials[4]  = one_trial()

#okay this is kinda jank but it works way better

In [8]:
df = pd.DataFrame(five_trials)
df

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
0,"(0, 0)","(0, 1)","(1, 1)","(0, 1)","(0, 2)","(0, 1)","(1, 1)","(2, 1)","(2, 2)","(2, 1)",...,None,None,None,None,None,None,None,None,None,None
1,"(0, 0)","(1, 0)","(0, 0)","(0, 1)","(0, 2)","(0, 1)","(0, 2)","(0, 3)","(1, 3)","(1, 4)",...,None,None,None,None,None,None,None,None,None,None
2,"(2, 1)","(1, 1)","(1, 0)","(1, 1)","(0, 1)","(0, 0)","(0, 1)","(0, 0)","(1, 0)","(1, 1)",...,"(0, 0)","(0, 1)","(1, 1)","(2, 1)","(2, 2)","(2, 3)","(2, 4)","(2, 3)","(3, 3)","(3, 4)"
3,"(0, 0)","(1, 0)","(1, 1)","(1, 0)","(0, 0)","(0, 1)","(0, 2)","(1, 2)","(0, 2)","(1, 2)",...,None,None,None,None,None,None,None,None,None,None
4,"(0, 0)","(0, 1)","(0, 0)","(0, 1)","(0, 2)","(0, 1)","(0, 0)","(1, 0)","(0, 0)","(1, 0)",...,None,None,None,None,None,None,None,None,None,None


In [9]:
df.to_csv("5_trials.csv")

In [38]:
#Time to develop the transfer model
#I'm gonna load this bad boy into a data frame
#P will be a 24x24 matrix for the 24 states
start = [0]*24
col = [(0,0),(0,1),(0,2),(0,3),(0,4),(0,5),(1,0),(1,1),(1,2),(1,3),(1,4),(1,5),(2,0),(2,1),(2,2),(2,3),(2,4),(2,5),(3,0),(3,1),(3,2),(3,3),(3,4),(3,5)]
transition = { 
    tuple((0,0)):[], 
    tuple((0,1)):[],    
    tuple((0,2)):[],
    tuple((0,3)):[],
    tuple((0,4)):[],
    tuple((0,5)):[],
    tuple((1,0)):[],
    tuple((1,1)):[],
    tuple((1,2)):[],
    tuple((1,3)):[],
    tuple((1,4)):[],
    tuple((1,5)):[],
    tuple((2,0)):[],
    tuple((2,1)):[],
    tuple((2,2)):[],
    tuple((2,3)):[],
    tuple((2,4)):[],
    tuple((2,5)):[],
    tuple((3,0)):[],
    tuple((3,1)):[],
    tuple((3,2)):[],
    tuple((3,3)):[],
    tuple((3,4)):[],
    tuple((3,5)):[],
    }

In [ ]:
five_trials[1]

In [43]:
g = (0,1)
column = col.index(g)
N = 1
N_all = 2

transition[g][column] = N/ N_all

In [33]:
for h in trials:
    print(h[1])

(0, 1)
(0, 1)
(1, 0)


In [35]:
trials = [0]*3
trials[0] = [(0,0),(0,1),(0,2),(1,2),(2,2)]
trials[1] = [(0,0),(0,1),(1,1),(0,1),(0,2),(1,2),(2,2)]
trials[2] = [(0,0),(1,0),(0,0),(0,1),(0,2),(1,2),(2,2)]


In [39]:
def find_indices(list_to_check, item_to_find):
    indices = []
    for idx, value in enumerate(list_to_check):
        if value == item_to_find:
            indices.append(idx)
    return indices




#This works for one trial, but the append method won't work for mulitple trials
for i in transition:
    s_prime = i #Chossing row
    for g in col:
        N = 0 #Everytime the column shifts, everything should revert
        N_all = 0
        s = g #did we end up with this value from s_prime?
        for h in five_trials:
            num_inst = find_indices(h,g) #Number of times s was in a trial, for which the list of trials is iterated through
            N_all += len(num_inst)
            for m in num_inst:
                actual_s_prime = h[m-1] #theres a bug here at pos (0,0), that manages to avoid getting triggered in the enxt step
                if actual_s_prime == s_prime:
                    N +=1 #Number of times in any trial that s came before s prime
        #Now we have N and N_all, now we need to input N/ N_all into the proper space in the transition model
        column = col.index(g)
        if N_all != 0:
            transition[tuple(i)].append(N/ N_all) #This should work bc we're iterating through g in order
        else:
            transition[tuple(i)].append(0)
        


NameError: name 'five_trials' is not defined

In [37]:
df2 = pd.DataFrame(transition, index=col)
df2.to_csv("transition.csv")